In [1]:
import tensorflow as tf

In [2]:
env = gym.make('Breakout-v4')
env

DependencyNotInstalled: No module named 'atari_py'. (HINT: you can install Atari dependencies by running 'pip install gym[atari]'.)